In [2]:
from selenium import webdriver
from undetected_chromedriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time
import sys
import string
import re
import unicodedata
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from lxml import html
import traceback
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from MySQLdb import _mysql
import datetime
import random


options = ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.headless=False
driver = Chrome(options=options, version_main = 122)

time.sleep(random.uniform(2,6))

db=_mysql.connect("localhost","root","","crawler_db")

for _ in range(10):
    try:
        db.query("CALL PROCEDURE_SEDAR(@p,@p1);")
        db.query('SELECT @p AS `SR_NO`,@p1 AS `KEYWORD`') 
        r=db.store_result()
        results=r.fetch_row()
        SR_NO = results[0][0].decode()
        KEYWORD = results[0][1].decode()
    
        print('SR_NO :', SR_NO, 'KEYWORD :', KEYWORD)
        break
        # keyword = 10003136
    except Exception as e: 
        print('Empty keyword / Deadlock issue', e)
        continue 

page = 1

driver.get("https://www.sedarplus.ca/landingpage/")
wait = WebDriverWait(driver, 30)
search_sedar = wait.until(EC.element_to_be_clickable((By.XPATH,"//a[text()='Search SEDAR+']" )))
search_sedar.click()
# time.sleep(2)
profile_option =  WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//li[@class="appTabFirst"]/a[@class = "keepInteractiveOnSubmit noFocus"]/span[@class = "appSpaces0 appLines1"]')))
driver.execute_script("arguments[0].click();",profile_option)

input_field = wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@id="QueryString" and @type="text"]')))
input_field.send_keys(KEYWORD)

profile_type = wait.until(EC.element_to_be_clickable((By.XPATH, '//textarea[@class="select2-search__field"]')))
profile_type.send_keys('Company')
profile_type.send_keys(Keys.RETURN)
time.sleep(2)
search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[span[text()="Search"]]')))
driver.execute_script("arguments[0].click();",search_button)




for j in range(1,4):

    for i  in range(1,11):
        
        def click_element_by_xpath(xpath):
            try:               
                test = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
                test.click()
                driver.set_page_load_timeout(5)
            except (TimeoutError, StaleElementReferenceException):
                print("TimeoutError or StaleElementReferenceException occurred. Trying to re-locate and click.")
                time.sleep(2)
                click_element_by_xpath(xpath)

        link_xpath =f'(//a[@class="searchIndustryParticipant-tabs-criteriaAndButtons-results-page-Entities-actionsBox-generateUrlBox-generateUrlBtn negativeLink appReadOnly appIndex0"])[{i}]'
        click_element_by_xpath(link_xpath)

        
        try:
            
            popup_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '//div[@class="ui-dialog ui-corner-all ui-widget ui-widget-content ui-front app-csa-party app-null appDialogRoot ui-draggable"]'))
            )
            
            html_string = popup_element.get_attribute('outerHTML')
            url_pattern = re.compile(r'value="(https://www\.sedarplus\.ca/csa-party/records/profile\.html\?id=[^"]+)"')
            matches = url_pattern.findall(html_string)

        except Exception as e:
            print(f"An error occurred: {e}")

        if matches:
            print(matches[0])
        else:
            print("No URL found.")
            continue

        time.sleep(5)      
        cancel_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//a[@class="searchIndustryParticipant-tabs-criteriaAndButtons-results-page-Entities-actionsBox-generateUrlBox-generateUrlDialog-buttons-cancel appCancel negativeLink appReadOnly appIndex0"]')))
        driver.execute_script("arguments[0].click();",cancel_button)

        driver.switch_to.new_window()
        driver.get(matches[0])
        driver.set_page_load_timeout(5)
        html_content1 = driver.page_source

        parsed_html = html.fromstring(html_content1)
        details_xpath_expression = '//span[text()="Regulatory actions"]'
        matching_elements = parsed_html.xpath(details_xpath_expression)

        if matching_elements:
            
            for i in range(3):
                time.sleep(2)
                regulatory_xpath = ['//span[text()="Regulatory actions"]' , '//a[.="Regulatory actions"]', '//span[@class="appSpaces1 appLines1" and text()="Regulatory actions"]']
                regulatory_action = wait.until(EC.element_to_be_clickable((By.XPATH, regulatory_xpath[i])))
                regulatory_action.click()

                # driver.set_page_load_timeout(5)
                regulatory_html = driver.page_source
                print("After clicking on 'default'")
                parsed_html = html.fromstring(regulatory_html)
                xpath_expression = '//div[@class="appRecordTitle appBlockTitle"]'
                matching_elements = parsed_html.xpath(xpath_expression)

                cease_xpath_expression = '//div[@class="appRecordTitle appBlockTitle appPullLeft" and h2 = "Cease trade orders"]'
                matching_elements_cease = parsed_html.xpath(cease_xpath_expression)
                if matching_elements_cease:
                    break       

                elif matching_elements:
                    break
                else:
                    continue

            if matching_elements:
                for i in range(2):
                    try:
                        time.sleep(2)
                        print("Cease order")
                        cease_xpath = [ '(//span[text()="Cease trade orders"])[2]', '(//a[.="Cease trade orders"])[2]', '//a[@class="keepInteractiveOnSubmit noFocus"]/span[@class="appSpaces2 appLines1"]' ]
                        cease_order =  driver.find_element(By.XPATH, cease_xpath[i])
                        print(cease_xpath[i])
                        driver.execute_script("arguments[0].click();",cease_order)
                        driver.implicitly_wait(5)
                        html_content3 = driver.page_source
                        print("After clicking on 'default'")
                        parsed_html = html.fromstring(html_content3)
                        xpath_expression = '//div[@class="appRecordTitle appBlockTitle appPullLeft" and h2 = "Cease trade orders"]'
                        matching_elements_cease_order = parsed_html.xpath(xpath_expression)
                        if matching_elements_cease_order:
                            break
                        else:
                            continue
                    except:
                        print(traceback.print_exc())
                        pass

                for _ in range(5):
                    try:
                        time.sleep(2)
                        print("discipilnary_action")
                        discipilnary_action =   wait.until(EC.element_to_be_clickable((By.XPATH,"//span[text() = 'Disciplinary actions']" )))
                        driver.execute_script("arguments[0].click();",discipilnary_action)
                        driver.implicitly_wait(5)
                        html_content4 = driver.page_source
                        print("After clicking on 'default'")
                        parsed_html = html.fromstring(html_content4)
                        xpath_expression = '//div[@class="appRecordTitle appBlockTitle appPullLeft" and h2 = "Disciplinary actions"]'
                        matching_elements_cease = parsed_html.xpath(xpath_expression)
                        if matching_elements_cease:
                            break
                        else:
                            continue
                    except:
                        print(traceback.print_exc())
                        pass

                for _ in range(5):
                    try:
                        time.sleep(2)
                        print("temporary_orders")
                        temporary_orders =  wait.until(EC.element_to_be_clickable((By.XPATH,"//span[text()='Temporary orders']")))
                        driver.execute_script("arguments[0].click();",temporary_orders)
                        driver.implicitly_wait(5)
                        html_content5 = driver.page_source
                        print("After clicking on 'default'")
                        parsed_html = html.fromstring(html_content5)
                        xpath_expression = '//div[@class="appRecordTitle appBlockTitle appPullLeft" and h2 = "Temporary orders"]'
                        matching_elements_cease = parsed_html.xpath(xpath_expression)
                        if matching_elements_cease:
                            break
                        else:
                            continue
                    except:
                        print(traceback.print_exc())
                        pass
                
                time.sleep(2)
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                

                html_page =html_content1 + regulatory_html + html_content3 + html_content4+ html_content5
                soup = BeautifulSoup(html_content1, 'html.parser')
                desired_element = soup.find('span', class_='appPageTitleText')

                if desired_element:
                    text_content = desired_element.text
                    print(text_content)
                else:
                    print('Element not found.')

                cleaned_text = re.sub(r'\s+', '_', text_content)
                cleaned_text = cleaned_text.replace('(', '').replace(')', '').replace(',','').replace('*', '_').strip('_')
                print(cleaned_text)

                with open(f"{cleaned_text}.html", "w", encoding="utf-8") as file:
                    file.write(html_page)

               
            elif matching_elements_cease:
                
                for _ in range(5):
                    try:
                        time.sleep(2)
                        print("discipilnary_action")
                        discipilnary_action =   wait.until(EC.element_to_be_clickable((By.XPATH,"//span[text() = 'Disciplinary actions']" )))
                        driver.execute_script("arguments[0].click();",discipilnary_action)
                        driver.implicitly_wait(5)
                        html_content4 = driver.page_source
                        print("After clicking on 'default'")
                        parsed_html = html.fromstring(html_content4)
                        xpath_expression = '//div[@class="appRecordTitle appBlockTitle appPullLeft" and h2 = "Disciplinary actions"]'
                        matching_elements_cease = parsed_html.xpath(xpath_expression)
                        if matching_elements_cease:
                            break
                        else:
                            continue
                    except:
                        print(traceback.print_exc())
                        pass


                for _ in range(5):
                    try:
                        time.sleep(2)
                        print("temporary_orders")
                        temporary_orders =  wait.until(EC.element_to_be_clickable((By.XPATH,"//span[text()='Temporary orders']")))
                        driver.execute_script("arguments[0].click();",temporary_orders)
                        driver.implicitly_wait(5)
                        html_content5 = driver.page_source
                        print("After clicking on 'default'")
                        parsed_html = html.fromstring(html_content5)
                        xpath_expression ='//div[@class="appRecordTitle appBlockTitle appPullLeft" and h2 = "Temporary orders"]'
                        matching_elements_cease = parsed_html.xpath(xpath_expression)
                        if matching_elements_cease:
                            break
                        else:
                            continue
                    except:
                        print(traceback.print_exc())
                        pass
            

                time.sleep(2)
                driver.close()
                driver.switch_to.window(driver.window_handles[0])  
                
                
                html_page =html_content1+ regulatory_html + html_content4 + html_content5
                soup = BeautifulSoup(html_content1, 'html.parser')
                desired_element = soup.find('span', class_='appPageTitleText')

                if desired_element:
                    text_content = desired_element.text
                    print(text_content)
                else:
                    print('Element not found.')

                cleaned_text = re.sub(r'\s+', '_', text_content)
                cleaned_text = cleaned_text.replace('(', '').replace(')', '').replace(',','').replace('*', '_').strip('_')

                with open(f"{cleaned_text}.html", "w", encoding="utf-8") as file:
                    file.write(html_page)

            
            else:
                print("check your Regulatory Xpath")

        else:
            # driver.find_element_by_xpath('//body[@class="appBody  app-csa-party"]').send_keys(Keys.CONTROL + 'w')
            time.sleep(2)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            

            html_page =html_content1
            soup = BeautifulSoup(html_content1, 'html.parser')
            desired_element = soup.find('span', class_='appPageTitleText')

            if desired_element:
                text_content = desired_element.text
                print(text_content)
            else:
                print('Element not found.')

            cleaned_text = re.sub(r'\s+', '_', text_content)
            cleaned_text = cleaned_text.replace('(', '').replace(')', '').replace(',','').replace('*', '_').strip('_')

            with open(f"{cleaned_text}.html", "w", encoding="utf-8") as file:
                file.write(html_page)
    
    # soup = (//div[@class="appPage pagination-item"])[last()]
    Next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '(//a[text()="Next »"])[1]')))
    for _ in range(5):
        try:
            db.query(f"UPDATE KEYWORDS_3_SEDAR SET PAGE_NO = {page} WHERE KEYWORD = '{KEYWORD}';")
            print("keyword updated")
            break
        except:
            time.sleep(2)
            continue
    page+=1
    driver.execute_script("arguments[0].click();",Next_button)




C:\Users\sujwa\AppData\Local\Temp\ipykernel_22912\693258072.py:28: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless=False


SR_NO : 6 KEYWORD : 5
https://www.sedarplus.ca/csa-party/records/profile.html?id=65cc75329d23d30e6c629b4a9fbf8b89
After clicking on 'default'
After clicking on 'default'
Cease order
(//span[text()="Cease trade orders"])[2]
After clicking on 'default'
Cease order
(//a[.="Cease trade orders"])[2]
After clicking on 'default'
discipilnary_action
After clicking on 'default'
discipilnary_action
After clicking on 'default'
temporary_orders
After clicking on 'default'
temporary_orders
After clicking on 'default'

                  
                    
                    
                      01 Communique Laboratory Inc. (000010658)
                    
                  
                  
                
01_Communique_Laboratory_Inc._000010658
https://www.sedarplus.ca/csa-party/records/profile.html?id=5ef4df295ea6da7a651beaa5379f5826
After clicking on 'default'
After clicking on 'default'
Cease order
(//span[text()="Cease trade orders"])[2]
After clicking on 'default'
Cease order
(//a[.=

TimeoutException: Message: timeout: Timed out receiving message from renderer: 0.222
  (Session info: chrome=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x008E8D03+51395]
	(No symbol) [0x00855F61]
	(No symbol) [0x0070E13A]
	(No symbol) [0x006FEE71]
	(No symbol) [0x006FECA3]
	(No symbol) [0x006FD98B]
	(No symbol) [0x006FE1CD]
	(No symbol) [0x00708E6A]
	(No symbol) [0x00717101]
	(No symbol) [0x0071A976]
	(No symbol) [0x006FE5F8]
	(No symbol) [0x00716F9F]
	(No symbol) [0x0077A87B]
	(No symbol) [0x00763C26]
	(No symbol) [0x0073C629]
	(No symbol) [0x0073D40D]
	GetHandleVerifier [0x00C668D3+3712147]
	GetHandleVerifier [0x00CA5CBA+3971194]
	GetHandleVerifier [0x00CA0FA8+3951464]
	GetHandleVerifier [0x00999D09+776393]
	(No symbol) [0x00861734]
	(No symbol) [0x0085C618]
	(No symbol) [0x0085C7C9]
	(No symbol) [0x0084DDF0]
	BaseThreadInitThunk [0x76CA7BA9+25]
	RtlInitializeExceptionChain [0x7733BDAB+107]
	RtlClearBits [0x7733BD2F+191]


In [3]:
pip install lxml


  Using cached lxml-5.1.0-cp310-cp310-win_amd64.whl.metadata (3.6 kB)
Using cached lxml-5.1.0-cp310-cp310-win_amd64.whl (3.9 MB)
